In [11]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2
import tensorflow
import keras
import keras_retinanet

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
#model_path = os.path.join('..', 'snapshots', 'resnet50_coco_best_v2.1.0.h5')
model_path = os.path.join('resnet50_coco_best_v2.1.0.h5')
# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
#model = models.convert_model(model)

#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 
                   6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 
                   12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 
                   19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 
                   25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 
                   31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 
                   36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 
                   41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 
                   48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 
                   54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 
                   60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 
                   66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 
                   72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 
                   78: 'hair drier', 79: 'toothbrush'}
# use this to change which GPU to use
#gpu = 0

# set the modified tf session as backend in keras
#setup_gpu(gpu)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


AttributeError: 'Node' object has no attribute 'output_masks'

In [2]:
from os.path import exists

git_repo_url = 'https://github.com/fizyr/keras-retinanet'
retina_net_dir = 'keras-retinanet'

if not exists(retina_net_dir):
    # clone "depth 1" will only get the latest copy of the relevant files.
    !git clone -q --recurse-submodules --depth 1 $git_repo_url
    # build
    !cd {retina_net_dir} && pip install .



Defaulting to user installation because normal site-packages is not writeable
Processing /home/hatsune/cv/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-1.0.0-cp38-cp38-linux_x86_64.whl size=176794 sha256=4040db927d84acde0e1362c70a0f181a16591ae9b8c8fcdc237e57cce466c21a
  Stored in directory: /home/hatsune/.cache/pip/wheels/1b/cd/fb/94692aea6f0a1ece9b843f767bbe9c2a1d142efc9433834a94
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20486 sha256=4ac586135d33290a30ec37b6e20b2fda0b17f6a0a65313473705afc8c1e8a02a
  Stored in directory: /home/hatsune/.cache/pip/wheels/be/90/98/9d455f04a7ca277366b36c660c89d171ff5abb7bdd8a8b8e75
Successfully built keras-retinanet keras-resnet
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [10]:
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import argparse
import imutils
import time
import cv2
import subprocess
import os
from playsound import playsound
import tensorflow
import keras
from keras.models import *
from keras.layers import *

cv2.setUseOptimized(True)
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

COLORS = np.random.uniform(0, 255, size=6)
#vs = VideoStream(0).start()
#vs = VideoStream("http://10.1.30.40:8080/video").start()
time.sleep(3.0)
fps = FPS().start()
y = 0
# loop over the frames from the video stream
kl_ = 0
key = ""
input_shape = (255,255,3)
num_classes = 2
imagelist = ""
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

#ansver_list = read file

for image in imagelist:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixels
    #frame = vs.read()
    #frame = imutils.resize(frame, width=720)
    #(height, width) = frame.shape[:2]
    # Let's get the starting pixel coordiantes (top left of cropped top)
    #start_row, start_col = int(0), int(0)
    # Let's get the ending pixel coordinates (bottom right of cropped top)
    #end_row, end_col = int(height * .5), int(width)
    #cropped_top = frame[start_row:end_row , start_col:end_col]
    #print (start_row, end_row)
    #print (start_col, end_col)

    im = image
    ss.setBaseImage(im)
    ss.switchToSelectiveSearchFast()
    rects = ss.process()
    imOut = im.copy()
    for i, rect in (enumerate(rects)):
        x, y, w, h = rect
        #     print(x,y,w,h)
        #     imOut = imOut[x:x+w,y:y+h]
        img = imOut[x:x+w,y:y+h]
        #cv2.rectangle(imOut, (x, y), (x+w, y+h), (0, 255, 0), 1, cv2.LINE_AA)
        # суём в нейронку и сравниваем с положительными результатами...
        true_rect = ansver_list[kl_].split(",")
        if ((x == true_rect[0] )|(x > true_rect[0] & x < true_rect[0] + 10)):
            if ((y == true_rect[0])|(y > true_rect[0] & y < true_rect[0] + 10)):
                if ((h == true_rect[0] )|(h > true_rect[0] & h < true_rect[0] + 10)):
                    if ((w == true_rect[0] )|(w > true_rect[0] & w < true_rect[0] + 10)):
                        # значит правдивый результат
                        pass
    # plt.figure()
    #plt.imshow(imOut)
    
    
    #cv2.rectangle(frame, (0, 0), (150, 100), COLORS[0], 2)
    #cv2.rectangle(frame, (0, 250), (150, 350), COLORS[2], 2)

    # Let's get the starting pixel coordiantes (top left of cropped bottom)
    #start_row, start_col = int(0), int(0)
    # Let's get the ending pixel coordinates (bottom right of cropped bottom)
    #end_row, end_col = int(150), int(100)
    #cropped_bot = frame[start_row:end_row , start_col:end_col]
    #print (start_row, end_row)
    #print (start_col, end_col)
    

    #cv2.imshow("Frame", frame)
    
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

    # update the FPS counter
    #fps.update()
    kl_ += 1
    #if (kl_ >= 10):
    #    kl_ = 0
    #    os.system('clear')

# stop the timer and display FPS information
#fps.stop()
#print("[INFO] elapsed time: {:.2f}".format(fps.elapsed()))
#print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

# do a bit of cleanup
#cv2.destroyAllWindows()
#vs.stop()

#cv2.imshow("Cropped Top", cropped_top) 
#cv2.waitKey(0) 
cv2.destroyAllWindows()

In [1]:
from imutils import paths
import argparse
import requests
import cv2
import os

# construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-u", "--urls", required=True,
	help="path to file containing image URLs")
ap.add_argument("-o", "--output", required=True,
	help="path to output directory of images")
args = vars(ap.parse_args())
# grab the list of URLs from the input file, then initialize the
# total number of images downloaded thus far
rows = open(args["urls"]).read().strip().split("\n")
total = 0

# loop the URLs
for url in rows:
	try:
		# try to download the image
		r = requests.get(url, timeout=60)
		# save the image to disk
		p = os.path.sep.join([args["output"], "{}.jpg".format(
			str(total).zfill(8))])
		f = open(p, "wb")
		f.write(r.content)
		f.close()
		# update the counter
		print("[INFO] downloaded: {}".format(p))
		total += 1
	# handle if any exceptions are thrown during the download process
	except:
		print("[INFO] error downloading {}...skipping".format(p))
# loop over the image paths we just downloaded
for imagePath in paths.list_images(args["output"]):
	# initialize if the image should be deleted or not
	delete = False
	# try to load the image
	try:
		image = cv2.imread(imagePath)
		# if the image is `None` then we could not properly load it
		# from disk, so delete it
		if image is None:
			delete = True
	# if OpenCV cannot load the image then the image is likely
	# corrupt so we should delete it
	except:
		print("Except")
		delete = True
	# check to see if the image should be deleted
	if delete:
		print("[INFO] deleting {}".format(imagePath))
		os.remove(imagePath)

In [ ]:
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import argparse
import imutils
import time
import cv2
import subprocess
import os
from playsound import playsound


COLORS = np.random.uniform(0, 255, size=6)
vs = VideoStream(0).start()
#vs = VideoStream("http://10.1.30.40:8080/video").start()
time.sleep(3.0)
fps = FPS().start()
y = 0
# loop over the frames from the video stream
kl_ = 0
while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=720)
    (height, width) = frame.shape[:2]
    # Let's get the starting pixel coordiantes (top left of cropped top)
    #start_row, start_col = int(0), int(0)
    # Let's get the ending pixel coordinates (bottom right of cropped top)
    #end_row, end_col = int(height * .5), int(width)
    #cropped_top = frame[start_row:end_row , start_col:end_col]
    #print (start_row, end_row)
    #print (start_col, end_col)

    cv2.rectangle(frame, (0, 0), (150, 100), COLORS[0], 2)
    cv2.rectangle(frame, (0, 250), (150, 350), COLORS[2], 2)

    # Let's get the starting pixel coordiantes (top left of cropped bottom)
    start_row, start_col = int(0), int(0)
    # Let's get the ending pixel coordinates (bottom right of cropped bottom)
    end_row, end_col = int(150), int(100)
    cropped_bot = frame[start_row:end_row , start_col:end_col]
    #print (start_row, end_row)
    #print (start_col, end_col)
    

    cv2.imshow("Frame", frame)
    
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

    # update the FPS counter
    fps.update()
    kl_ += 1
    if (kl_ >= 10):
        kl_ = 0
        os.system('clear')

# stop the timer and display FPS information
fps.stop()
print("[INFO] elapsed time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

# do a bit of cleanup
#cv2.destroyAllWindows()
vs.stop()

#cv2.imshow("Cropped Top", cropped_top) 
#cv2.waitKey(0) 
cv2.destroyAllWindows()